In [ ]:
#IE 360 HW1
#SEFA KAMBUR


# Load the readxl package
library(readxl)
library(lubridate)
library(zoo)
library(dplyr)
library(ggplot2)
library(forecast)

# Prompt user to select the xls file using a file dialog
file_path <- file.choose()

# Read the xls file into R
promotion_data  <- read_excel(file_path)


# View the structure of the data
str(promotion_data)


# Check for missing values in the time series data
missing_values <- sum(is.na(promotion_data))
print(paste("Number of missing values:", missing_values))

# Set meaningful column names for promotion_data
colnames(promotion_data) <- c("Date", "Promotion_Trend")

# Convert the 'Date' column to Date type
promotion_data$Date <- as.Date(promotion_data$Date)

# Extract month from the 'Date' column
promotion_data$Month <- floor_date(promotion_data$Date, "month")

# Aggregate the 'Promotion_Trend' column by month
monthly_promotion <- promotion_data %>%
  group_by(Month) %>%
  summarize(Promotion_Trend = sum(Promotion_Trend))

# View the structure of the aggregated data
str(monthly_promotion)
monthly_promotion


# Check for missing values
missing_values <- colSums(is.na(monthly_promotion))
# Print the number of missing values
print(missing_values)

# Create a time series plot 
ggplot(monthly_promotion, aes(x = Month, y = Promotion_Trend)) +
  geom_line() +
  labs(x = "Month", y = "Promotion Trend", title = "Monthly Promotion Trend") +
  theme_minimal()

# Create a bar plot of monthly promotion trend
ggplot(monthly_promotion, aes(x = format(Month, "%b %Y"), y = Promotion_Trend)) +
  geom_bar(stat = "identity", fill = "skyblue") +
  labs(x = "Month", y = "Promotion Trend", title = "Monthly Promotion Trend") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))  # Rotate x-axis labels for better readability


# Create a time series object 
ts_monthly_promotion <- ts(monthly_promotion$Promotion_Trend, start = c(2018, 1), frequency = 12)
ts_monthly_promotion

# Read the data from the selected Excel file
file_path <- file.choose()
data <- read_excel(file_path)

# Check 
head(data)

# Verify if the dataset contains numerical data
str(data)

# Convert the Tarih column to Date format
data$Tarih <- as.Date(paste0(data$Tarih, "-01"), format = "%Y-%m-%d")

# Check for missing values in the data
missing_values <- sum(is.na(data))
# Print the number of missing values
print(paste("Number of missing values in data:", missing_values))

# Merge the two datasets based on the date column
merged_data <- merge(monthly_promotion, data, by.x = "Month", by.y = "Tarih", all = TRUE)

# View the structure of the merged data
str(merged_data)

# Convert the merged_data data frame to a time series object
ts_merged_data <- ts(merged_data[, -1], start = c(2018, 1), frequency = 12)

# Print the structure of the ts_merged_data
str(ts_merged_data)
ts_merged_data

# Fit linear regression model with corrected variable names
lm_model <- lm(Promotion_Trend ~ `Credit Card Expenses` + `Kartlı Ödeme Endeksi` + `TP FG J03` + `TP KARTMETRE D1`, data = ts_merged_data)

# Summary of the model
  summary(lm_model)


# Load the necessary libraries
library(ggplot2)
library(car)

# Residuals vs Fitted Values plot
plot1 <- ggplot(lm_model, aes(.fitted, .resid)) +
  geom_point() +
  geom_hline(yintercept = 0, linetype = "dashed", color = "red") +
  labs(x = "Fitted Values", y = "Residuals", title = "Residuals vs Fitted Values")

# Normal Q-Q plot
plot2 <- ggplot(lm_model, aes(sample = .stdresid)) +
  stat_qq() +
  stat_qq_line() +
  labs(title = "Normal Q-Q Plot")

plot2

# Scale-Location plot
plot3 <- ggplot(lm_model, aes(.fitted, sqrt(abs(.stdresid)))) +
  geom_point() +
  geom_smooth() +
  labs(x = "Fitted Values", y = "Square Root of Standardized Residuals", title = "Scale-Location Plot")

plot3



# Generate predictions using the linear regression model
predictions <- predict(lm_model)

# Create a new data frame to store actual and predicted values
predicted_data <- data.frame(
  Month = rownames(ts_merged_data),  # Assuming Month is the row names of your original data
  Actual = ts_merged_data$Promotion_Trend,
  Predicted = predictions
)

# Plot actual vs predicted values
ggplot(predicted_data, aes(x = Month)) +
  geom_line(aes(y = Actual, color = "Actual")) +
  geom_line(aes(y = Predicted, color = "Predicted")) +
  labs(title = "Actual vs Predicted Promotion Trend") +
  scale_color_manual(values = c("Actual" = "blue", "Predicted" = "red")) +
  theme_minimal()


# Convert 
ts_merged_data <- data.frame(ts_merged_data)

# Apply the log transformation
ts_merged_data$log_Promotion_Trend <- log(ts_merged_data$Promotion_Trend + 1)  # Adding 1 to handle zero values


str(ts_merged_data)

# Fit linear regression model with log-transformed 
lm_model_log <- lm(log_Promotion_Trend ~ `Credit.Card.Expenses` + `Kartlı.Ödeme.Endeksi` + `TP.FG.J03` + `TP.KARTMETRE.D1`, data = ts_merged_data)

# Summary of the model
summary(lm_model_log)

